In [ ]:
client_id = 
client_secret = 

import pandas as pd
from datetime import datetime, timedelta
import requests
import re
def handle_authorization(data):
    
    access_token = data["access_token"]
    print(f" {access_token}")
    return access_token
    
def get_twitch_authorization(client_id, client_secret):
    url = f"https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
    }
    
    response = requests.post(url, params=params)
    data = response.json()
    
    handle_authorization(data)



# Obtén la autorización de Twitch
get_twitch_authorization(client_id, client_secret)

In [ ]:
oauth_token = 'zjjwht0e4fl6o2ptep00tq86azvfhk'

In [ ]:
def extract_channel_name(url):
    pattern = r"https?:\/\/(?:www\.|go\.)?twitch\.tv\/([a-zA-Z0-9_]{4,25})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None
def get_streamer_data(client_id, url):
    channel_name = extract_channel_name(url)
    if not channel_name:
        return None
    
    url = f"https://api.twitch.tv/helix/users"
    
    params = {
        "login": channel_name
    }
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    print(data)
    if "data" in data and len(data["data"]) > 0:
        user_id = data["data"][0]["id"]
        stream_url = f"https://api.twitch.tv/helix/streams?user_id={user_id}"
        response = requests.get(stream_url, headers=headers)
        stream_data = response.json()
        
        if "data" in stream_data and len(stream_data["data"]) > 0:
            return stream_data["data"][0]
    
    return None
url = "https://www.twitch.tv/kingsleague"
streamer_data = get_streamer_data(client_id, url)

if streamer_data:
    data = f"Datos del streamer {channel_url}:"

In [ ]:
data = {'data': [{'id': '121606712', 'login': 'kingsleague', 'display_name': 'kingsleague', 'type': '', 'broadcaster_type': 'partner', 'description': 'Kings League InfoJobs I La liga de los reyes. Queens League Oysho I La liga de las reinas. ', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/cc310a6e-8cee-435b-af4c-5b480b791037-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/2aa7b955-7f20-4fa9-8b32-d8ee40474f68-channel_offline_image-1920x1080.png', 'view_count': 0, 'created_at': '2016-04-12T13:00:54Z'}]}
data_dict = data['data'][0]
data_dict
id_canal =  list(data_dict.values())[0]
id_canal

In [ ]:
def obtener_vods(id_canal):
    # Calcular la fecha hace un año desde hoy
    fecha_inicio = datetime.now() - timedelta(days=730)
    fecha_inicio_str = fecha_inicio.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100"

    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }

    vods_data = []

    while url:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            vods = data["data"]

            for vod in vods:
                vod_id = int(vod["id"])
                fecha_inicio = datetime.strptime(vod["created_at"], "%Y-%m-%dT%H:%M:%SZ")
                fecha_final = datetime.strptime(vod["published_at"], "%Y-%m-%dT%H:%M:%SZ")
                duracion = vod["duration"]
                visitas = int(vod["view_count"])
                nombre = vod["title"].lower().replace(' ', '_')
                descripcion = vod["description"]
                url_vod = f"https://www.twitch.tv/videos/{vod_id}"

                vods_data.append({
                    "ID del VOD": vod_id,
                    "URL del VOD": url_vod,
                    "Nombre del VOD": nombre,
                    "Duración": duracion,
                    "Fecha de inicio": fecha_inicio,
                    "Fecha de finalización": fecha_final,
                    "Visitas": visitas,
                    
                })

            pagination = data.get("pagination", None)
            if pagination and "cursor" in pagination:
                cursor = pagination["cursor"]
                url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100&after={cursor}"
            else:
                url = None

        else:
            print("Se produjo un error al realizar la solicitud GET.")
            return None

    # Crear un DataFrame a partir de los datos
    df_vods = pd.DataFrame(vods_data)

    return df_vods

df = obtener_vods(id_canal)
df.info(memory_usage='deep')
df.head()

In [ ]:
df.to_csv('twitch_api_kingsleague_csv.csv', index=False)